<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 15px;">

## Linear Regression and Object Oriented Programming

Week 3 | 3.1

---

### Deriving the least squares solution to linear regression

With target vector $y$ and prediction matrix $X$, we can formulate a regression as:

### $$ y = \beta X + \epsilon $$

Where $\beta$ is our vector of coefficients and $\epsilon$ is our vector of errors, or residuals.

We can equivalently formulate this as a calculation of the residuals:

### $$ \epsilon = \beta X - y $$

Our goal is to minimize the sum of squared residuals. The sum of squared residuals is equivalent to the dot product of the vector of residuals:

### $$ \sum_{i=1}^n \epsilon_i^2 = 
\left[\begin{array}{cc}
\epsilon_1 \cdots \epsilon_n
\end{array}\right] 
\left[\begin{array}{cc}
\epsilon_1 \\ \cdots \\ \epsilon_n
\end{array}\right] = \epsilon' \epsilon
$$

Therefore we can write the sum of squared residuals as:

### $$ \epsilon' \epsilon = (\beta X - y)' (\beta X - y) $$

Which becomes:

### $$ \epsilon' \epsilon = y'y - y'X\beta - \beta' X' y + \beta' X' X \beta $$

Now take the derivative with respect to $\beta$:

### $$ \frac{\partial \epsilon' \epsilon}{\partial \beta} = 
-2X'y + 2X'X\beta$$

We want to minimize the sum of squared errors, and so we set the derivative to zero and solve for the beta coefficient vector:

### $$ 0 = -2X'y + 2X'X\beta \\
X'X\beta = X'y \\
\beta = (X'X)^{-1}X'y$$

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

house = '/Users/kiefer/github-repos/DSI-SF-2/datasets/housing_data/housing-data.csv'
house = pd.read_csv(house)

In [23]:
tmp = np.array([[1,2],[2,5]])
tmp

array([[1, 2],
       [2, 5]])

In [24]:
np.linalg.inv(tmp)

array([[ 5., -2.],
       [-2.,  1.]])

---

## Classes and Objects

In python, everything is an "object" of a specific type. This is the basis of what is known as object oriented programming.

A class is a type of object. You can think of a class definition as a sort of "blueprint" that specifies the construction of a new object when instantiated.

Knowing how to define and use classes is esential to programming python at an intermediate or advanced level. I will cover the basics here, which will help you understand how thinks like LinearRegression in sklearn work.



---

### Coding a simple version of `LinearRegression`

By now you are familiar with the `LinearRegression` class in sklearn. We will walk through the re-creation of this class (a simplified version).


### 1. The class definition

Below is the beginning of our class blueprint:

In [2]:
class SimpleLinearRegression(object):
    
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

What are the components of this?

**`class`**

- The `class` is like `def`, but instead of defining a function it defines a class.

**`object`**

- `object` in the parentheses of the class definition indicate that this class "inherits" from the `object` class. The object class is a very general, very fundamental class in python. Inheritance means that whatever properties and function are part of the `object` class are passed down to our `SimpleLinearRegression` class.

**`def __init__(self)`**

- The `def __init__(self):` is our class's initialization function. This function is called when you instantiate the class by typing `SimpleLinearRegression()`

**`self`**

- `self` is the (confusing) first argument to class definitions. It is a variable that refers to the **current instantiation of the class**. What does this mean? When you instantiate a class and assign it to a variable with `slr = SimpleLinearRegression()`, the `self` argument is now a reference to the current instantiation of the class `slr`. Now, when you use a function that is part of the class, it knows to use that specific object's function. This lets you have multiple instantiations of a class with the same function name.

**class attributes**

- `self.coef_` and `self.intercept_`, likewise, are "attributes" (variables) that are connected to the instantiation of the class. When self becomes `slr`, for example, the `self` becomes `slr` and `self.coef_` becomes `slr.coef`

---

### 2. Adding a class function

Now, just like with `__init__`, we can add functions to the class.

Let's add a `calculate_betas()` method that will calculate the coefficients for a linear regression.

In [3]:
class SimpleLinearRegression(object):
    
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self, X, y):
        # betas formula
        # betas = (X'X)^-1 X'Y
        
        XtX = np.dot(X.T, X)
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = np.dot(XtX_inv, X.T)
        self.coef_ = np.dot(XtX_inv_Xt, y)

Notice that we assigned `self.coef_` inside of the `calculate_betas()` function.

This will set the class attribute `self.coef_`, and this attribute can be accessed by _any other function in the class without passing it as an argument!_

It can also be accessed by you after instantiating the class.

---

### 3. Assigning attributes during instantiation

There is an issue here - we may pass an `X` matrix in without an intercept. 

Add a keyword argument to the `__init__` function which will specify whether the `X` matrix should have an intercept added or not.

In [4]:
class SimpleLinearRegression(object):
    
    def __init__(self, fit_intercept=True):
        self.coef_ = None
        self.intercept_ = None
        self.fit_intercept = fit_intercept
        
    def fit(self, X, y):
        # betas formula
        # betas = (X'X)^-1 X'Y
        
        XtX = np.dot(X.T, X)
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = np.dot(XtX_inv, X.T)
        self.coef_ = np.dot(XtX_inv_Xt, y)
        

Now, if we instantiate the class, it will assign `fit_intercept` to the class attribute `fit_intercept`, like so:

In [5]:
slr = SimpleLinearRegression(fit_intercept=True)
slr.fit_intercept

True

In [6]:
slr = SimpleLinearRegression(fit_intercept=False)
slr.fit_intercept

False

---

### 4. Add a function to add an intercept to the `X` matrix if necessary

This function will be called from inside the `fit` function.

In [7]:
class SimpleLinearRegression(object):
    
    def __init__(self, fit_intercept=True):
        self.coef_ = None
        self.intercept_ = None
        self.fit_intercept = fit_intercept
        
    def add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        X = np.concatenate([intercept, X], axis=1)
        return X
        
    def fit(self, X, y):
        
        if self.fit_intercept:
            X = self.add_intercept(X)
        
        # betas formula
        # betas = (X'X)^-1 X'Y
        
        XtX = np.dot(X.T, X)
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = np.dot(XtX_inv, X.T)
        betas = np.dot(XtX_inv_Xt, y)
        
        self.coef_ = betas[1:]
        self.intercept_ = betas[0]

---

### 5. Try out the class...

Let's instantiate the class and try out the coefficient fitting function on the housing data.

In [8]:
y = house.price.values
X = house[['sqft','bdrms','age']].values

In [9]:
slr = SimpleLinearRegression(fit_intercept=True)
print slr.fit_intercept
print slr.coef_
print slr.intercept_

True
None
None


In [10]:
slr.fit(X, y)

In [11]:
print slr.coef_
print slr.intercept_

[  139.33484671 -8621.47045953   -81.21787764]
92451.6278416


Like in the sklearn `LinearRegression` class, we now have access to the assigned `coef_` and `intercept_` attributes after fitting the model.

---

### 6. Add the `predict` function.

Let's add some more of the class methods that are in the real `LinearRegression` class.

First off add the `predict` function. It will take a design matrix `X` and return predictions for those rows.

In [12]:
class SimpleLinearRegression(object):
    
    def __init__(self, fit_intercept=True):
        self.coef_ = None
        self.intercept_ = None
        self.fit_intercept = fit_intercept
        
    def add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        X = np.concatenate([intercept, X], axis=1)
        return X
        
    def fit(self, X, y):
        
        if self.fit_intercept:
            X = self.add_intercept(X)
        
        # betas formula
        # betas = (X'X)^-1 X'Y
        
        XtX = np.dot(X.T, X)
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = np.dot(XtX_inv, X.T)
        betas = np.dot(XtX_inv_Xt, y)
        
        self.coef_ = betas[1:]
        self.intercept_ = betas[0]
        
    def predict(self, X):
        if self.fit_intercept:
            X = self.add_intercept(X)
            
        return np.dot(X, np.concatenate([[self.intercept_], self.coef_]))

In [13]:
slr = SimpleLinearRegression(fit_intercept=True)
slr.fit(X,y)
y_hat = slr.predict(X)

In [14]:
print y.shape, y_hat.shape

(47,) (47,)


---

### 7. Add a `score` method.

This will calculate the $R^2$ of your model on a provided `X` and `y`.

You'll probably need to write a helper function to calculate the sum of squared errors, since this will be run for both the baseline model and the regression model in order to calculate the $R^2$.

In [15]:
class SimpleLinearRegression(object):
    
    def __init__(self, fit_intercept=True):
        self.coef_ = None
        self.intercept_ = None
        self.fit_intercept = fit_intercept
        
    def add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        X = np.concatenate([intercept, X], axis=1)
        return X
        
    def fit(self, X, y):
        
        if self.fit_intercept:
            X = self.add_intercept(X)
        
        # betas formula
        # betas = (X'X)^-1 X'Y
        
        XtX = np.dot(X.T, X)
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = np.dot(XtX_inv, X.T)
        betas = np.dot(XtX_inv_Xt, y)
        
        self.coef_ = betas[1:]
        self.intercept_ = betas[0]
        
    def predict(self, X):
        if self.fit_intercept:
            X = self.add_intercept(X)
            
        return np.dot(X, np.concatenate([[self.intercept_], self.coef_]))
    
    def _calculate_sse(self, y_true, y_hat):
        return np.sum((y_true - y_hat)**2)
        
    def _calculate_r2(self, sse_model, sse_baseline):
        return 1. - float(sse_model)/sse_baseline
    
    def score(self, X, y):
            
        baseline_sse = self._calculate_sse(y, np.tile(np.mean(y), len(y)))
        
        y_hat = self.predict(X)
        model_sse = self._calculate_sse(y, y_hat)
        
        return self._calculate_r2(model_sse, baseline_sse)
          

In [16]:
slr = SimpleLinearRegression(fit_intercept=True)
slr.fit(X,y)
r2 = slr.score(X, y)
print r2

0.733163999069


Check against sklearn's implementation:

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

//anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


0.73316399906900243

---

### 8. Inspecting a class

When we want to know more about a class object, we can use the "inspect" module. Specifically the `inspect.getmembers()` function takes an instantiated class as an argument and returns an information dictionary.

This can be helpful to know what attributes and methods are avaiable and basically, the blueprint of a class object in memory.  Depending on the way the class was implemented, you can usually find useful information hiding inside of `slr.__class__.__dict__` -- which can be easier to look at.  The "right way" is to use the "inspect" module.

In [19]:
import inspect

In [20]:
inspect.getmembers(slr)

[('__class__', __main__.SimpleLinearRegression),
 ('__delattr__',
  <method-wrapper '__delattr__' of SimpleLinearRegression object at 0x1178da050>),
 ('__dict__',
  {'coef_': array([  139.33484671, -8621.47045953,   -81.21787764]),
   'fit_intercept': True,
   'intercept_': 92451.627841645211}),
 ('__doc__', None),
 ('__format__', <function __format__>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of SimpleLinearRegression object at 0x1178da050>),
 ('__hash__',
  <method-wrapper '__hash__' of SimpleLinearRegression object at 0x1178da050>),
 ('__init__',
  <bound method SimpleLinearRegression.__init__ of <__main__.SimpleLinearRegression object at 0x1178da050>>),
 ('__module__', '__main__'),
 ('__new__', <function __new__>),
 ('__reduce__', <function __reduce__>),
 ('__reduce_ex__', <function __reduce_ex__>),
 ('__repr__',
  <method-wrapper '__repr__' of SimpleLinearRegression object at 0x1178da050>),
 ('__setattr__',
  <method-wrapper '__setattr__' of SimpleLinearRegressi

---

### Special Class Methods

|Method| Description|
|--|--|
|\_\_init\_\_ ( self [,args...] )| Constructor (with any optional arguments) Sample Call : obj = className(args)
|\_\_del\_\_( self ) | Destructor, deletes an object Sample Call : del obj
|\_\_repr\_\_( self ) | Evaluatable string representation Sample Call : repr(obj)
|\_\_str\_\_( self ) | Printable string representation Sample Call : str(obj)
|\_\_cmp\_\_ ( self, x ) | Object comparison Sample Call : cmp(obj, x)

The `__repr__` function reports back something descriptive about what the class represents.  You can basically do whatever you want with it but the purpose of it is to convey something descirptive about your class.

The `__del__` method is the bookend function of `__init__`. You can use it to run code once your class is done executing.  

Generally it works well but in practice there are a few things watch out for.  Read more about [safely using Python destructors](http://eli.thegreenplace.net/2009/06/12/safely-using-destructors-in-python)